## Introdução




O City of New York é uma plataforma de dados abertos que centraliza informações sobre serviços e atividades na cidade de Nova Iorque, incluindo o amplo conjunto de reclamações feito por cidadãos por meio do sistema 311. Para este projeto, definiu-se como objetivo comparar os 10 mil primeiros e os 10 mil últimos registros disponíveis, a fim de verificar possíveis diferenças entre as reclamações ocorridas em um período inicial e em um período recente. Dessa maneira, torna-se viável observar como o perfil das reclamações pode ter se transformado ao longo do tempo.

## 1. Coleta de dados


Neste trabalho, a etapa de obtenção dos dados foi bastante complicada, não em relação à codificação em si, mas sim ao método de acesso às informações. Não foi possível realizar o download dos dados no site oficial, de modo que se optou por empregar requisições à API do City of New York. Contudo, essa API impõe uma limitação que disponibiliza apenas mil registros por requisição. Para contornar essa restrição, desenvolveu-se uma função de coleta, capaz de fazer múltiplas chamadas e agregar os resultados em um único DataFrame.

### Importando libs necessárias

In [ ]:
!pip install ipywidgets plotly

import requests
import pandas as pd
from urllib.parse import urlencode
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px


### Função para requisição de dados

A função `fetch_nyc311_data` faz requisições à API do NYC 311 para coletar um certo número de registros (definido por `total_records`). Como a API retorna, no máximo, 1.000 resultados por requisição, o código precisa efetuar várias chamadas sequenciais até alcançar o total desejado.

- **Parâmetros**:
  - `total_records`: quantidade total de linhas que se deseja obter (padrão: 5.000).
  - `order`: critério de ordenação (por exemplo, `"created_date ASC"` ou `"created_date DESC"`).

- **Lógica**:
  1. Define o endpoint da API e inicia variáveis para controle de paginação (`offset`, `limit_`, `fetched`).
  2. Enquanto não atingir `total_records` baixados, monta os parâmetros de consulta, incluindo quantos registros (`$limit`) e a posição (`$offset`).
  3. Caso `order` seja fornecido, adiciona-o à query string (`$order`).
  4. Faz a requisição e adiciona os resultados à lista `data_list`.
  5. Atualiza os contadores (`fetched` e `offset`) para a próxima iteração.
  6. Se a chamada não retornar mais dados ou ocorrer erro, o loop encerra.
  7. Ao final, converte `data_list` em um `pandas.DataFrame` e o retorna.


In [ ]:
def fetch_nyc311_data(total_records=5000, order=None):
    base_url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"
    data_list = []
    offset = 0
    limit_ = 1000
    fetched = 0
    while fetched < total_records:
        remaining = total_records - fetched
        batch = min(limit_, remaining)
        params = {"$limit": batch, "$offset": offset}
        if order:
            params["$order"] = order
        qs = urlencode(params)
        url = f"{base_url}?{qs}"
        resp = requests.get(url)
        if resp.status_code == 200:
            chunk = resp.json()
            if not chunk:
                break
            data_list.extend(chunk)
            fetched += len(chunk)
            offset += len(chunk)
        else:
            break
    return pd.DataFrame(data_list)

## Utilizando a função para buscar dados


In [ ]:
df_oldest = fetch_nyc311_data(total_records=10000, order="created_date ASC")
df_oldest["segment"] = "oldest"

df_newest = fetch_nyc311_data(total_records=10000, order="created_date DESC")
df_newest["segment"] = "newest"

df_all = pd.concat([df_oldest, df_newest], ignore_index=True).dropna(subset=["complaint_type"])
df_all["complaint_type"] = df_all["complaint_type"].astype(str)

## 2. Criação do Gráfico

A ideia é criar um gráfico vendo a cmoparação de todas as reclamações e poder escolher uma tipo de reclamação para ver de forma individual

### Criação do gráfico inicial

In [ ]:
grouped_all = df_all.groupby(["complaint_type","segment"]).size().reset_index(name="count")

fig_default = px.bar(
    grouped_all,
    x="complaint_type",
    y="count",
    color="segment",
    barmode="group",
    title="Comparação de Todos os Tipos (Oldest vs Newest)"
)

### Configuração dos widgets

In [ ]:
all_types = sorted(df_all["complaint_type"].unique())

multi_select = widgets.SelectMultiple(
    options=all_types,
    description="Filtrar Tipo:",
    rows=10,
    layout=widgets.Layout(width='50%')
)

###  Função de callback

In [ ]:
def on_button_click(b):
    with output:
        output.clear_output()
        selected = multi_select.value
        if not selected:
            print("Nenhum tipo selecionado. Exibindo todos.")
            filtered = df_all
        else:
            filtered = df_all[df_all["complaint_type"].isin(selected)]
        if filtered.empty:
            print("Sem dados para esse filtro.")
            return
        grouped_filtered = filtered.groupby(["complaint_type","segment"]).size().reset_index(name="count")
        fig_filtered = px.bar(
            grouped_filtered,
            x="complaint_type",
            y="count",
            color="segment",
            barmode="group",
            title="Comparação Filtrada (Oldest vs Newest)"
        )
        fig_filtered.update_layout(xaxis_title="Complaint Type", yaxis_title="Count")
        fig_filtered.show()


### Display Final

In [ ]:
button = widgets.Button(description="Aplicar Filtro")
output = widgets.Output()

fig_default.update_layout(xaxis_title="Complaint Type", yaxis_title="Count")
fig_default.show()
button.on_click(on_button_click)
display(multi_select, button, output)

SelectMultiple(description='Filtrar Tipo:', index=(109,), layout=Layout(width='50%'), options=('APPLIANCE', 'A…

Button(description='Aplicar Filtro', style=ButtonStyle())

Output()

## 3. Análise Geral

A comparação dos primeiros 10 mil registros de reclamações e dos últimos 10 mil evidencia uma mudança significativa em diversos tipos de queixa ao longo do tempo. Observa-se que:

- **Algumas categorias** praticamente não aparecem no conjunto antigo, mas despontam com intensidade no grupo mais recente.  
- **Outras reclamações** permanecem relativamente constantes ou mesmo diminuem.  
- É provável que **novas formas de categorização**, bem como a maior facilidade de registro tenham influenciado o volume de reclamações.  

Destacam-se dois temas específicos que se sobressaem nessa comparação.


### 1) Noise - Residential

A reclamação de barulho residencial aparece com grande peso na parte mais recente dos dados. Possíveis motivos:

- **Conscientização**: Mais pessoas podem estar cientes de que podem reportar incômodos sonoros à prefeitura.  
- **Mudanças urbanas**: Áreas com maior densidade populacional ou crescimento de atividades noturnas tendem a gerar mais ruído.  
- **Classificação aprimorada**: O sistema 311 pode ter passado a categorizar melhor o tipo de barulho, elevando oficialmente as estatísticas de “Noise - Residential”.


### 2) Illegal Parking

O aumento de reclamações referentes a estacionamento ilegal indica uma maior percepção desse problema por parte da população ou um real agravamento do trânsito. Fatores como:

- **Frota de veículos em expansão**  
- **Serviços de delivery e aplicativos**  
- **Rotinas de trabalho flexíveis**


## 4. Conclusão

A análise entre registros antigos e recentes revela tendências que podem ser exploradas em maior profundidade, considerando variáveis como localização, horários e sazonalidade. Compreender essas transformações ajuda a embasar políticas públicas e estratégias de fiscalização, além de orientar cidadãos e organizações comunitárias quanto aos problemas mais frequentes no ambiente.